# Gemma 7B Instruct QLoRA Fine Tune

This Gemma model fine-tuned in another enviroment, then imported to Kaggle. The first cell below gets the necessary files from Kaggle. You can ignore it if you use a notebook from Kaggle.

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T124356Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0e42726848e5dc421f4806827168c3dfc45eee2b9cf8d9c75009b47ba661c2c3729adbee6fa9982575c2df637c85289738b1b9f23b7a01531c11f3866d34d1c250b524800465b969c23a6da7951c81ebac407a6b8d79ab7f8077f21ca35351f368d4f63c60d0d1264ad0ef9012fa6822654b904f4ac3b3668a7c580e3b5606a755744cee24a58847957801c2756d4c758441bcab47245b059b36535eb909abf9cb2ec3f471f32086df0bb530ef7df24223b285a9a2c58d8eddfc2dd8fe4c8e603ed075c737c1ee84afde9992550964a3369c935cd847c2bdb58e3273e51487868a3e0a8fd29a2e31ec0d1928d7cd80e83ba0214fd89fab9fe046c94bc59afbc0,gemma/transformers/7b-it/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F8332%2F11394%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T124357Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D84791282594a647fdb1f698f45c8882e474209f32b51a2aa31b6b7afb8f65802f9ef908dc620970288041755bf709b5802c91ccb31022c6fe04ca2c166c75cfc551eaea498a2bad83a06e26c7cd5cbdfa626e8edd470658784704a71721fd12e970a166d22dd6762150212f772cf3788d47719aabcc00919e0af841fd8f3c8d62e96ccffba9b6b9aff0213b9fd071a44ec318631fae32a7f444b2a26d64a3038e7ca60301088e18d7df57111d236b19f54681c3a1e3990850b906af9e687196fe0b3e2faad4e046302176d9944a52c7ec13e1ad65819af723f67a85e21b2a969ebdeefda8fbb995b5ce9632411a5fbecf8d240fe1536e2dba6e307e045c9b587'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


### First, we install necessary libraries for fine-tuning, and datasets.

In [2]:
!pip install transformers datasets accelerate peft trl bitsandbytes wandb












































     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 799.9 kB/s eta 0:00:00 0:00:01





































































   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.8 MB/s eta 0:00:00a 0:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.5 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 11.3 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.9 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00:00:0100:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.4 MB/s eta 0:00:0000:01:00:01



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.4 MB/s eta 0:00:000:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 12.1 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━

Hugging Face Transformers and Datasets libraries are already installed in Kaggle Notebooks. However, when I process the dataset, I encountered an error and updating the Datasets library solved the problem. Also, Gemma models comes with latest HF Transformers Library, so updating both Transformers and Datasets is a must.

In [4]:
!pip install transformers datasets --upgrade




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 754.1 kB/s eta 0:00:00a 0:00:01




































   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 12.8 MB/s eta 0:00:0000:0100:01


  Attempting uninstall: transformers

    Found existing installation: transformers 4.37.2

    Uninstalling transformers-4.37.2:

      Successfully uninstalled transformers-4.37.2





[notice] A new release of pip is available: 23.3.2 -> 24.0

[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np # linear algebra
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-assistants-with-gemma/submission_instructions.txt

/kaggle/input/data-assistants-with-gemma/submission_categories.txt


In [6]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Google provides Gemma to users with disclaimer, so you must accept the disclaimer if you want to use Gemma from Hugging Face. Accept it and get a HF Token from your profile and enter it here.

In [7]:
from huggingface_hub import notebook_login
notebook_login()

# Data Preprocessing

Google wants these features answered or explained from Gemma:
* Data Science Basics and Concepts
* Python Basics and Concepts
* Kaggle Basics and Concepts

I did not found any dataset related to Kaggle platform, so I did not add any dataset related to Kaggle.
I found 2 dataset from HF related to out subject:

* https://huggingface.co/datasets/mlabonne/Evol-Instruct-Python-1k
* https://huggingface.co/datasets/RazinAleks/SO-Python_QA-Data_Science_and_Machine_Learning_class

Edit: I found this dataset for Kaggle, but it has 1M+ rows so it's HUGE. Maybe you will try :)

#### Plan for Preprocessing

As I said, I found 2 datasets. I will combine these 2 datasets as one and fine-tune Gemma with the combined dataset. It seemed more appropriate to me to do this. You can do this process seperated, if you wish.

Instruct dataset is Python code dataset, QA Dataset is question-answer dataset related to Python and Data Science concepts.

In [8]:
import datasets

rawInstructDataset = datasets.load_dataset("mlabonne/Evol-Instruct-Python-1k", split="train")
rawQADataset = datasets.load_dataset("RazinAleks/SO-Python_QA-Data_Science_and_Machine_Learning_class")
rawInstructDataset, rawQADataset

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.

  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

(Dataset({
     features: ['instruction', 'output'],
     num_rows: 1000
 }),
 DatasetDict({
     train: Dataset({
         features: ['ViewCount', 'CreationDate', 'Answer', 'Tags', 'Available Count', 'Q_Score', 'Networking and APIs', 'Q_Id', 'Score', 'Data Science and Machine Learning', 'Database and SQL', 'Other', 'GUI and Desktop Applications', 'Users Score', 'A_Id', 'Title', 'is_accepted', 'AnswerCount', 'Web Development', 'Python Basics and Environment', 'Question', 'System Administration and DevOps'],
         num_rows: 6223
     })
     validation: Dataset({
         features: ['ViewCount', 'CreationDate', 'Answer', 'Tags', 'Available Count', 'Q_Score', 'Networking and APIs', 'Q_Id', 'Score', 'Data Science and Machine Learning', 'Database and SQL', 'Other', 'GUI and Desktop Applications', 'Users Score', 'A_Id', 'Title', 'is_accepted', 'AnswerCount', 'Web Development', 'Python Basics and Environment', 'Question', 'System Administration and DevOps'],
         num_rows: 1778
     }

Both datasets structures are different. So we must play with them a little bit. I will refactor QA dataset's structure like Instruct dataset. 

Let's start with Instruct dataset. I will combine question and answer as one column named "instructions".

In [9]:
instructTexts = []

for instruction, output in zip(rawInstructDataset["instruction"], rawInstructDataset["output"]):
    instructText = instruction + output
    instructTexts.append(instructText)
    
instructDataset = rawInstructDataset
instructDataset = instructDataset.add_column("instructions", column=instructTexts)
instructDataset = instructDataset.remove_columns(["instruction", "output"])
instructDataset

Dataset({
    features: ['instructions'],
    num_rows: 1000
})

Continue the same process with QA Dataset, but merge train, test and validation.

In [10]:
from datasets import concatenate_datasets

trainQADataset = rawQADataset["train"]
testQADataset = rawQADataset["test"]
validationQADataset = rawQADataset["validation"]

rawCombinedQADataset = concatenate_datasets([trainQADataset, validationQADataset, testQADataset])
rawCombinedQADataset

Dataset({
    features: ['ViewCount', 'CreationDate', 'Answer', 'Tags', 'Available Count', 'Q_Score', 'Networking and APIs', 'Q_Id', 'Score', 'Data Science and Machine Learning', 'Database and SQL', 'Other', 'GUI and Desktop Applications', 'Users Score', 'A_Id', 'Title', 'is_accepted', 'AnswerCount', 'Web Development', 'Python Basics and Environment', 'Question', 'System Administration and DevOps'],
    num_rows: 8889
})

There are lots of unnecessary columns. Delete them and combine questions and answers as one column.

In [11]:
QATexts = []

for question, answer in zip(rawCombinedQADataset["Question"], rawCombinedQADataset["Answer"]):
    qaText = question + answer
    QATexts.append(qaText)
    
QADataset = rawCombinedQADataset
QADataset = QADataset.add_column("instructions", column=QATexts)
QADataset = QADataset.remove_columns(['Q_Score', 'Networking and APIs', 'Available Count', 'CreationDate', 'System Administration and DevOps', 
                                      'GUI and Desktop Applications', 'Database and SQL', 'Other', 'ViewCount', 'Score', 'Tags', 'Web Development', 
                                      'Data Science and Machine Learning', 'AnswerCount', 'A_Id', 'Title', 'is_accepted', 'Answer', 'Q_Id', 'Question', 
                                      'Python Basics and Environment', 'Users Score'])
QADataset

Dataset({
    features: ['instructions'],
    num_rows: 8889
})

Finally, merge all final datasets as one final dataset.

In [12]:
finalDataset = concatenate_datasets([instructDataset, QADataset])
finalDataset

Dataset({
    features: ['instructions'],
    num_rows: 9889
})

# Model Preparation and Fine-Tuning

Gemma is a big model (especially 7b) for our hardwares. To eliminate this, we use some techniques, such as quantization and LoRA (Low-Rank Adaptation). You can find detailed info [here](https://pytorch.org/blog/finetune-llms/). Let's prepare our both LoRA and quantization configs below:

In [13]:
from peft import LoraConfig
from transformers import BitsAndBytesConfig

loraConfig = LoraConfig(
    r = 8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

modelName = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForCausalLM.from_pretrained(modelName, quantization_config=bnb_config, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

I will use Trainer for training, and used this trainer arguments from Google's Gemma QLoRA fine-tuning script [here](https://huggingface.co/google/gemma-7b/blob/main/examples/example_sft_qlora.py).

In [15]:
from transformers import TrainingArguments

trainArgs = TrainingArguments(
    report_to="wandb",
    output_dir="devfiles/results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=20,
    logging_steps=20,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=100,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    gradient_checkpointing=True,
    fp16=False,
    bf16=False)

NOTE: I tried training Gemma with 2xT4 GPU's, but VRAM (2x16) was not enough. So, I used a RTX A6000 (48GB) for training from another enviroment.

In [16]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    args = trainArgs,
    train_dataset = finalDataset,
    peft_config = loraConfig,
    packing = True,
    dataset_text_field = "instructions",
    tokenizer = tokenizer
)
trainer.train()

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024

  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.

  warnings.warn(

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.

  warnings.warn(

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

  warnings.warn(


Step,Training Loss
20,5.838300
40,1.948800
60,1.750600
80,1.668500
100,1.642200


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

  warnings.warn(

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

  warnings.warn(

/root/miniconda3/envs/py3.10/lib/python3.10/site-p

TrainOutput(global_step=100, training_loss=2.5696884536743165, metrics={'train_runtime': 5528.1434, 'train_samples_per_second': 0.289, 'train_steps_per_second': 0.018, 'total_flos': 7.6443656650752e+16, 'train_loss': 2.5696884536743165, 'epoch': 0.44})

Our model has trained, and ready to use. Let's try Gemma with examples:

In [20]:
device = "cuda:0"
questions = ["How do i create an array and assign values in Python?", "What are Hierarchical Bayes models and where are they used?"]

for question in questions:
    inputs = tokenizer(question, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=64)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))




How do i create an array and assign values in Python?



You can create an array in Python using the square brackets [] and assign values to it using the square brackets [] as well.



Here is an example:



```python

arr = [1, 2, 3, 4, 5]

print(arr)

```



Output:





What are Hierarchical Bayes models and where are they used?



Hierarchical Bayes models are a type of Bayesian model that are used to model complex data structures. They are often used in situations where there is a lot of data and the data is not necessarily independent.



Hierarchical Bayes models are often used in situations where there is a lot of data and the data is not necessarily independent.


As you can see, model works fine. You can try whatever prompt you want.

If you want, you can save the model

In [21]:
model.save_pretrained("devfiles/saved-models")